<a href="https://colab.research.google.com/github/PavelZhuravkov/sf_data_science/blob/main/%D0%94%D0%B8%D0%BF%D0%BB%D0%BE%D0%BC%D0%BD%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%D0%9A%D0%BE%D0%BC%D0%BF%D0%B8%D0%BB%D1%8F%D1%86%D0%B8%D1%8F_%D0%B4%D0%B0%D1%82%D0%B0%D1%81%D0%B5%D1%82%D0%B0_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<center>ОПРЕДЕЛЕНИЕ УЯЗВИМЫХ ГРУПП НАСЕЛЕНИЯ </center>

![ 'fjf'](https://nacto.org/wp-content/uploads/2016/04/Universal-Design-02.jpg)

In [ ]:
import numpy as np
import pandas as pd
import pickle as pkl
import re

##Данные для исследованй взяты из открытых источников. Основным поставщиком сведений является государственное статистическое ведомство " РОССТАТ".
##Выборки будем делать исходя из самой свежей и полной информации,на сегодняшний день, а именно за 2022 год.

Не всегда один и тот же регион пишется одинаково в разных датасетах.
Такое очень часто происходит с написаниями регионов РФ в разных
документах. Это
необходимо учитывать при объединении датасетов.

Особенностью Тюменской области является то, что в её состав входят
два других субъекта Российской Федерации: Ханты-Мансийский
автономный округ — Югра и Ямало-Ненецкий автономный округ. При
этом чаще всего в датасетах указывается отдельно Тюменская область
и отдельно — без учёта автономных округов.
То же самое касается Архангельской области — в её состав входит
самостоятельный субъект Ненецкий автономный округ. Способы
написания такие же, как с Тюменской областью.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Десериализуем pipeline из файла
with open('/content/data/regions_rf.pkl', 'rb') as pkl_file:
    regions_rf = pkl.load(pkl_file)

In [ ]:
def get_region(arg):
  """ Функция для получения стандартных названий регионов РФ """
  for key, value in regions_rf.items():
    if re.search(key, arg):
        return value

In [ ]:
double_region = ['Архангельская область', 'Тюменская область'] # Список дублированных областей с учетом автономных округов

In [ ]:
def get_data(df):
  """ Функция для получения датасета """
  df = df.dropna(inplace = False).reset_index(drop = True) # Убираем значения nan
  df['region'] = df['region'].apply(lambda x: x.strip()) # Убираем лишние пробелы
  df = df[df['region'].isin (double_region) == False ] # Убираем дублтрованные области
  df['region'] = df['region'].apply(lambda x: x.lower()) # Переходим в нижний регистр
  df['region'] = df['region'].apply(lambda x: re.sub(r"\(.*\)", "", x)) # Убираем то, что в скобках
  df['region'] = df['region'].apply(get_region) # Запускаем функцию стандартных названий регионов
  df = df.dropna(inplace = False).reset_index(drop = True) # Избавляемся от значений None
  df = df.sort_values('region').reset_index(drop =True) # Сортируем по алфавиту датасет
  return df

#1.  Подготовка данных к объединению сводного DataFrame.

##1.0 Среднедушевые денежные доходы населения по субъектам Российской федерации, руб./месяц

Данные взяты на ресурсе " РОССТАТ"

Nominal_income - Номинальный доход на душу населения.

In [ ]:
df_0= pd.read_excel('/content/data/Urov_10subg-nm.xlsx') # Читаем датафрейм и переименовываем признаки за 2022 год
df_0 = df_0.rename(columns = {'Среднедушевые денежные доходы населения по субъектам \nРоссийской федерации, руб./месяц *': 'region', 'Unnamed: 50':'nominal_income'})
df_0 = df_0[['region', 'nominal_income']]
df_0 = get_data(df_0)
total_df = df_0

#1.1  Реальные денежные доходы населения по субъектам Российской федерации,%

Данные взяты на ресурсе " РОССТАТ"

real_income - реальные денежные доходы населения по субъектам РФ

In [ ]:
df_1 = pd.read_excel('/content/data/Urov_11subg-nm.xlsx')
df_1 = df_1.rename(columns = {'Unnamed: 45': 'real_income'})
df_1 = df_1[['region', 'real_income']]
df_1 = get_data(df_1)
total_df = total_df.merge(df_1, on = 'region', how = 'left')

##1.2 Среднемесячная номинальная начисленная заработная плата работников по полному кругу организаций по субъектам Российской Федерации за 12 месяцев 2022 года, рублей

Данные взяты на ресурсе " РОССТАТ"

avg_salary - Среднемесячная номинальная начисленная заработная плата работников по полному кругу организаций по субъектам Российской Федерации с 2019 года, рублей'

In [ ]:
df_2 = pd.read_excel('/content/data/tab2-zpl_07-2023.xlsx')
df_2 = df_2[df_2['region'].isin (double_region) == False] # Удаляем строки с Архангельской и Тюменской областями, которые вместе с округами
df_2.drop(df_2.iloc [:, 1:36], axis =1, inplace =True) # Оставляем признаки за 12 месяцев 2022 года.
df_2.drop(df_2.iloc[:, 13:26], axis = 1, inplace =True) # Удаляем признаки с данными за часть 2023 года.
df_2 = get_data(df_2)
df_2['avg_salary'] = round(df_2.iloc[:, 1:12].mean(axis =1)) # Находим среднюю зарплату по году.
df_2 = df_2[['region', 'avg_salary']]
total_df = total_df.merge(df_2, on = 'region', how ='left')

#1.3 Реальная начисленная заработная плата работников по полному кругу организаций по субъектам Российской Федерации за 12 месяцев 2022 года, %

Данные взяты на ресурсе " РОССТАТ"

real_avg_salary - Реальная начисленная заработная плата работников по полному кругу организаций по субъектам Российской Федерации за 12 месяцев 2022 года, %

In [ ]:
df_3 = pd.read_excel('/content/data/tab5-zpl_2022.xlsx')
df_3 = df_3[['region', 'Unnamed: 5']].rename(columns= {'Unnamed: 5': 'real_avg_salary'})
df_3 = get_data(df_3)
total_df = total_df.merge(df_3, on = 'region', how ='left')

#1.4 Детская смертность по субъектам РФ

Данные взяты на ресурсе " РОССТАТ".

infant_moratality - Число детей, умерших в возрасте до 1 года

infant_mortality_per_1000 - Число детей, умерших в возрасте до 1 года на 1000 родившихся.

In [ ]:
df_4 = pd.read_excel('/content/data/Edn_06-2023_t2.xlsx')
df_4 = df_4.rename(columns= {'Unnamed: 2': 'infant_mortality', 'Unnamed: 6': 'infant_mortality_per_1000'})
df_4 = df_4[['region', 'infant_mortality', 'infant_mortality_per_1000']]
df_4['region'] = df_4['region'].replace({'Hижегородская область': 'Нижегородская область', 'Hенецкий авт.округ': 'Ненецкий авт.округ'})
df_4 = get_data(df_4)
total_df = total_df.merge(df_4, on = 'region', how ='left')

#1.5 Число людей с инвалидностью
#по регионам, без учета детей-инвалидов на 01.01.2023 год.

Данные взяты на ресурсе " РОССТАТ"

disabled - Число людей с инвалидностью
по регионам, без учета детей-инвалидов

In [ ]:
df_5 = pd.read_excel('/content/data/pi_1.3.xlsx')
df_5 = df_5.rename(columns = {'Unnamed: 2': 'disabled'})
df_5 = df_5[['region', 'disabled']]
df_5 =get_data(df_5)
total_df = total_df.merge(df_5, on = 'region', how ='left')

#1.6  Заболеваемость на 100 тыс.
#человек населения, по возрастным группам и группам заболеваний

child_pregnancy - Детская беременность.

congenital_anomalies - Врожденные аномалии (пороки развития), деформации и хромосомные нарушения.

all_deseases - Все заболевания.



neoplasms -Новообразования.

mental_disorders -Психические расстройства и расстройства поведения.

In [ ]:
df_6 = pd.read_excel('/content/data/morbidity_2005_2020_age_disease.xls')
df_6 = df_6.rename(columns = {'Заболеваемость с диагнозом, установленным впервые в жизни, проживающих в районе обслуживания лечебного учреждения, на 100 тыс.человек населения (человек, на 1 января)':'region', 'Unnamed: 1': 'ind',	'Unnamed: 14': '2016'})
df_pregnans = df_6[(df_6['ind'] == 'Беременность, роды и послеродовой период') & (df_6['Unnamed: 2'] == '0-14 лет')] # Оставляем только детскую беременность
df_pregnans = df_pregnans[['region', 'ind', '2016']]
df_pregnans = get_data(df_pregnans)
df_pregnans = df_pregnans.pivot_table(values = '2016', index = 'region', columns = 'ind')
df_pregnans = df_pregnans.reset_index()

df_6 = df_6[df_6['Unnamed: 2'] == 'Всего']
df_6 = df_6[['region', 'ind', '2016']]
df_6 = df_6.pivot_table(values = '2016', index = 'region', columns = 'ind')
df_6 = df_6.reset_index()
df_6 = get_data(df_6)
df_6 = df_6.sort_values('region').drop(['Беременность, роды и послеродовой период'], axis =1).reset_index(drop =True)
df_6 = df_pregnans.merge(df_6, on = 'region', how = 'inner')
df_6 = df_6.rename(columns ={'Беременность, роды и послеродовой период': 'child_pregnancy',
                             'Врожденные аномалии (пороки развития), деформации и хромосомные нарушения': 'congenital_anomalies',
                             'Все заболевания': 'all_deseases',
                             'Новообразования': 'neoplasms',
                             'Психические расстройства и расстройства поведения': 'mental_disorders'})
df_6 = df_6[['region', 'child_pregnancy', 'congenital_anomalies', 'neoplasms', 'mental_disorders', 'all_deseases']]
total_df = total_df.merge(df_6, on = 'region', how ='left')

#1.7 Процент населения с доходами ниже прожиточного минимуиа( черта бедности) по субъектам РФ, %


Данные взяты на ресурсе " РОССТАТ"

percent_poverty - Процент населения с доходами ниже прожиточного минимуиа( черта бедности) по субъектам РФ, %

In [ ]:
df_7 = pd.read_excel('/content/data/tab2-6.xlsx')
df_7['percent_poverty'] = df_7['Unnamed: 2'] + df_7['Unnamed: 3']
df_7 = df_7[['region', 'percent_poverty']]
df_7 = get_data(df_7)
total_df = total_df.merge(df_7, on = 'region', how ='left')

#1.8 ОБЩАЯ ЧИСЛЕННОСТЬ НАСЕЛЕНИЯ ПО ОСНОВНЫМ ВОЗРАСТНЫМ ГРУППАМ
#ПО СУБЪЕКТАМ РОССИЙСКОЙ ФЕДЕРАЦИИ
#в среднем за 2022 г.

Данные взяты на ресурсе " РОССТАТ"

population - все наседение


population_childs - население модоже трудоспособного возраста


population_employmens - трудоспособное население

population_retired - население старше трудоспособного возраста

In [ ]:
df_8 = pd.read_excel('/content/data/Население  по регионам РФ по возрастам 2022.xlsx')
df_8 = df_8.rename(columns = {'Unnamed: 1': 'population', 'Unnamed: 4': 'population_childs', 'Unnamed: 7': 'population_employmens', 'Unnamed: 10': 'population_retired'})
df_8 = df_8[['region', 'population', 'population_childs', 'population_employmens', 'population_retired']]
df_8 = get_data(df_8)
total_df = total_df.merge(df_8, on = 'region', how ='left')

#1.9 Xарактеристика жилищных условий домохозяйств за 2022 год.


Данные взяты на ресурсе " РОССТАТ"

total_living_square_one_persone - Общая площадь жилых помещений, приходящаяся в среднем на одного жителя, м2

In [ ]:
df_9 = pd.read_excel('/content/data/jil_fond_2020-2022.xls')
df_9 = df_9.rename(columns = {'Unnamed: 4': 'total_living_square_one_persone'})
df_9 = df_9[['region', 'total_living_square_one_persone']]
df_9 = get_data(df_9)
total_df = total_df.merge(df_9, on = 'region', how ='left')

#1.10 Дети

Данные взяты с портала "Если быть точным".

poor_family - Малоимущие семьи с детьми, %

disabled_children - Дети с инвалидностью, %

orphans - Дети, родители к-рых лишены / ограничены в родительских правах, чел/10000

In [ ]:
df_10 = pd.read_excel('/content/data/report_childhood_2020.xlsx')
df_10 = df_10.rename(columns = {'Unnamed: 1': 'region','Unnamed: 2': 'poor_family', 'Unnamed: 3': 'disabled_children', 'Unnamed: 4': 'orphans'})
df_10 = df_10[['region', 'poor_family', 'disabled_children', 'orphans']]
df_10 = get_data(df_10)
total_df = total_df.merge(df_10, on = 'region', how ='left')

#1.11 Предоставление гражданам жилых помещений по субъектам Российской Федерации за 2022 г.

Данные взяты на ресурсе " РОССТАТ"

family_need_housing - Число семей, состоящих на учете в качестве нуждающихся в жилых помещениях на конец отчетного года , единиц

family_got_housing - Число семей получивших жилые помещения и улучшивших жилищные условия, единиц

specific_gravity_family - Удельный вес числа семей, получивших жилые помещения, в числе семей, состоящих на учете в качестве нуждающихся в жилых помещениях на конец прошлого года, в %

In [ ]:
df_11 = pd.read_excel('/content/data/Pred_jilpom_2020-2022.xls')
df_11 = df_11.rename(columns = {'Unnamed: 2': 'family_need_housing', 'Unnamed: 3': 'family_got_housing', 'Unnamed: 4': 'specific_gravity_family'})
df_11 = df_11.drop('Unnamed: 1', axis =1)
df_11 = get_data(df_11)
total_df = total_df.merge(df_11, on = 'region', how ='left')

#1.12 Количество семей, получивших жилое помещение по договорам социального найма в 2022 году, тыс. семей

Данные взяты на ресурсе " РОССТАТ"

social_housing_hiring - Количество семей, получивших жилое помещение по договорам социального найма, тыс. семей

In [ ]:
df_12 = pd.read_excel('/content/data/pred_jilpom_socnaym_08-2023.xlsx')
df_12 = df_12.rename(columns = {'Unnamed: 23': 'social_housing_hiring'})
df_12 = df_12[['region', 'social_housing_hiring']]
df_12 = get_data(df_12)
total_df = total_df.merge(df_12, on = 'region', how ='left')

#1.13  Сведения о заболеваемости ВИЧ, на 10
#тыс. обследованных за 2022 год.

Данные взяты с портала "Если быть точным".

hiv_patient - Выявлено на 10 тыс. обследованных

In [ ]:
df_13 = pd.read_excel('/content/data/report_hiv_2022.xlsx')
df_13 = df_13.rename(columns = {'Unnamed: 1': 'region', 'Unnamed: 3': 'hiv_patient'})
df_13 = df_13[['region', 'hiv_patient']]
df_13 =get_data(df_13)
total_df = total_df.merge(df_13, on = 'region', how ='left')

#1.14 Алкоголизм и наркомания данные за 2019 год.

Данные взяты с портала "Если быть точным".

alcohol_consumption - Потребление алкоголя на 1 человека 15+, литр/душу

drug_addiction - Общая заболеваемость (наркотики), на 100 тыс. населения.

In [ ]:
df_14 = pd.read_excel('/content/data/report_alkogolizm_i_narkomaniya_2019.xlsx')
df_14 = df_14.rename(columns ={'Unnamed: 1': 'region', 'Unnamed: 2': 'alcohol_consumption', 'Unnamed: 4': 'drug_addiction'})
df_14 = df_14[['region', 'alcohol_consumption', 'drug_addiction']]
df_14 = get_data(df_14)
total_df = total_df.merge(df_14, on = 'region', how ='left')

#1.15  Рождённые в этом месяце, по регионам, без
#учёта мертворождённых за 2022 год

count_newborn - количество новорожденных

count_newborn_per_1000 - количество новорожденных на 1000 населения.

In [ ]:
df_15 = pd.read_excel('/content/data/EDN_2022.xlsx')
df_15 = df_15.rename(columns ={'Unnamed: 1': 'count_newborn', 'Unnamed: 4': 'count_newborn_per_1000'})
df_15 = df_15[['region', 'count_newborn', 'count_newborn_per_1000']]
df_15 = get_data(df_15)
total_df = total_df.merge(df_15, on = 'region', how ='left')

#1.16 Распределение малоимущего населения по социально-демографическим группам за 2020 год, %

childrens  - Дети в возрасте до 16 лет

pensioners - Население старше трудоспособного возраста


employmens - Население трудоспособного возраста

In [ ]:
df_2020 = pd.read_excel('/content/data/poverty_socdem_2020.xls') # Прочитаем представленные данные
df_2019 = pd.read_excel('/content/data/poverty_socdem_2019.xls')
df_2018 = pd.read_excel('/content/data/poverty_socdem_2018.xls')
df_2017 = pd.read_excel('/content/data/poverty_socdem_2017.xls')

In [ ]:
def rename_columns(df): # Функция для переименования признаков и удаления ненужных.
  df = df.rename(columns = {'Unnamed: 2': 'childrens', 'Unnamed: 3': 'pensioners', 'Unnamed: 4': 'employmens'})
  df = df[['region', 'childrens', 'pensioners', 'employmens']]
  return df

In [ ]:
df_2020 = rename_columns(df_2020)
df_2019 = rename_columns(df_2019)
df_2018 = rename_columns(df_2018)
df_2017 = rename_columns(df_2017)
df_16 = df_2020

In [ ]:
df_16.isnull().sum() # Проверяем количество нулевых значений

region         2
childrens     13
pensioners    13
employmens    13
dtype: int64

In [ ]:
df_16 = df_16[(df_16[['childrens', 'pensioners', 'employmens']].isna().all(axis=1))] = df_2019 # Берем недостающие данные из датасета за 2019 год.
df_16.isnull().sum()


region        2
childrens     1
pensioners    1
employmens    1
dtype: int64

In [ ]:
df_16 = get_data(df_16)
total_df = total_df.merge(df_16, on = 'region', how ='left')

#1.17 Отношение числа занятых в экономике региона к численности населения регтона в трудоспособном возрасте, %

Данные взяты на ресурсе " РОССТАТ"

percrnt_employmens -  Отношение числа занятых в экономике региона к численности
населения региона в трудоспособном возрасте, %,

data_1 - Численность населения в трудоспособном возрасте по регионам РФ за 2022 год.

data_2 - Численность занятых в экономике по регионам РФ за 2022 год.

In [ ]:
data_1 = pd.read_excel('/content/data/Бюллетень_2023.xlsx')
data_1 = data_1[['region', 'Unnamed: 3']]
data_1 = get_data(data_1)
data_2 = pd.read_excel('/content/data/Trud-2_15-72.xlsx')
data_2 = data_2[['region', 'Unnamed: 23']]
data_2 = get_data(data_2)
df_17 = data_1.merge(data_2, on = 'region', how = 'inner')
df_17['percent_employmens'] = df_17['Unnamed: 23'] / df_17['Unnamed: 3'] * 100
df_17['percent_employmens'] = df_17['percent_employmens'].apply(lambda x: round(x))
df_17 = df_17[['region', 'percent_employmens']]
total_df = total_df.merge(df_17, on = 'region', how ='left')

#1.18 Валовый региональный продукт на душу населения за 2020 год, в рублях.

Данные взяты на ресурсе " РОССТАТ"

gross_regional_product - Валовый региональный продукт на душу населения за 2020 год, в рублях.

In [ ]:
df_18 = pd.read_excel('/content/data/RR_pokaz_09-02_2022.xlsx')
df_18 = df_18.rename(columns ={'Unnamed: 6': 'gross_regional_product'})
df_18 = df_18[['region', 'gross_regional_product']]
df_18 = get_data(df_18)
total_df = total_df.merge(df_18, on = 'region', how ='left')

#1.19 Объем отгруженных товаров собственного производства, выполненных работ и услуг собственными силами по субъектам Российской Федерации за 2022 год.
 (млн.руб.)

Данные взяты на ресурсе " РОССТАТ"

regional_product -  объём отгруженных товаров
собственного производства или работ/услуг, выполненных
собственными силами, за 2022 г. в тыс. рублей.

In [ ]:
sheet = pd.read_excel('/content/data/otgruz_sub_god (1).xlsx')
sheet_1 = pd.read_excel('/content/data/otgruz_sub_god (1).xlsx', sheet_name= 1)
sheet_2 = pd.read_excel('/content/data/otgruz_sub_god (1).xlsx', sheet_name= 2)
sheet_3 = pd.read_excel('/content/data/otgruz_sub_god (1).xlsx', sheet_name= 3)
sheet_4 = pd.read_excel('/content/data/otgruz_sub_god (1).xlsx', sheet_name= 4)

In [ ]:
def get_product_set(df):
  df = df.rename(columns = {'К содержанию': 'region'})
  df = df[['region', 'Unnamed: 6']]
  df = get_data(df)
  return df

In [ ]:
sheet = get_product_set(sheet)
sheet_1 = get_product_set(sheet_1)
sheet_2 = get_product_set(sheet_2)
sheet_3 = get_product_set(sheet_3)
sheet_4 = get_product_set(sheet_4)

In [ ]:
df_19 = sheet.merge(sheet_1, on = 'region').merge(sheet_2, on = 'region').merge(sheet_3, on = 'region').merge(sheet_4, on = 'region')
df_19['regional_product'] = df_19.iloc[:, 1:6].sum(axis =1)
df_19 = df_19[['region', 'regional_product']]
total_df = total_df.merge(df_19, on = 'region', how ='left')

#1.20 Оборот розничной
#торговли на душу населения за 2022 год, в рублях.

Данные взяты на ресурсе " РОССТАТ"

retail_turnover_per_capital - Оборот розничной торговли на душу населения, в рублях

In [ ]:
df_20 = pd.read_excel('/content/data/Oborot_dusha.xlsx')
df_20 = df_20.rename(columns = {'Unnamed: 23': 'retail_turnover_per_capital'})
df_20 = df_20[['region', 'retail_turnover_per_capital']]
df_20 = get_data(df_20)
total_df = total_df.merge(df_20, on = 'region', how ='left')

# Прожиточный минимум для трудоспособного населения за 2022 год.

liwing_wage - прожиточный минимум.

In [ ]:
df_21 = pd.read_excel('/content/data/прожиточный минимум.xlsx')
df_21 = df_21.rename(columns = {'В том числе по социально-демографическим группам:': 'living_wage'})
df_21 = df_21[['region', 'living_wage']]
df_21 = get_data(df_21)
total_df = total_df.merge(df_21, on = 'region', how = 'left')


#1.22 Расходы на социальную политику от
#общих расходов бюджета региона, % в год.

In [ ]:
def get_data(df):
  """ Функция для получения датасета """
  df = df.dropna(inplace = False).reset_index(drop = True) # Убираем значения nan
  df['region'] = df['region'].apply(lambda x: x.strip()) # Убираем лишние пробелы
  #df = df[df['region'].isin (double_region) == False ] # Убираем дублтрованные области
  df['region'] = df['region'].apply(lambda x: x.lower()) # Переходим в нижний регистр
  df['region'] = df['region'].apply(lambda x: re.sub(r"\(.*\)", "", x)) # Убираем то, что в скобках
  df['region'] = df['region'].apply(get_region) # Запускаем функцию стандартных названий регионов
  df = df.dropna(inplace = False).reset_index(drop = True) # Избавляемся от значений None
  df = df.sort_values('region').reset_index(drop =True) # Сортируем по алфавиту датасет
  return df

Данные взяты с портала "Если быть точным".

welfare_expense_share - Расходы на социальную политику от
общих расходов бюджета региона, % в год.

In [ ]:
df_22 = pd.read_excel('/content/data/welfare_expense_share_2015_2020.xlsx')
df_22 = df_22.rename(columns = {2020: 'welfare_expense_share'})
df_22 = df_22[['region', 'welfare_expense_share']]
df_22 = get_data(df_22)
total_df = total_df.merge(df_22, on = 'region', how ='left')

#1.23 Преступность

total_crime - Всего зарегистрированных преступлений.

especialy_serious_crime	- Количество особо тяжких преступлений.

serious_crime	- Количество тяжких преступлений.

crime_of_moderaty_severity- Количество преступлений средней тяжести.

minor_crime - Количество преступлений небольшой тяжести.

In [ ]:
def get_crime_set(df):
  df = df.rename(columns = {'Unnamed: 1': 'region'})
  df.drop('Unnamed: 0', axis = 1, inplace =True)
  df = get_data(df)
  df['total'] = df.iloc[:, 1:11].sum(axis =1).astype(int)
  df = df[['region', 'total']]
  return df

In [ ]:
sheet = pd.read_excel('/content/data/4-EGS_Razdel_4_122020.xls')
sheet_1 = pd.read_excel('/content/data/4-EGS_Razdel_4_122020.xls', sheet_name= 1)
sheet_2 = pd.read_excel('/content/data/4-EGS_Razdel_4_122020.xls', sheet_name= 2)
sheet_3 = pd.read_excel('/content/data/4-EGS_Razdel_4_122020.xls', sheet_name= 3)
sheet_4 = pd.read_excel('/content/data/4-EGS_Razdel_4_122020.xls', sheet_name= 4)


In [ ]:
sheet = get_crime_set(sheet)
sheet_1 = get_crime_set(sheet_1)
sheet_2 = get_crime_set(sheet_2)
sheet_3 = get_crime_set(sheet_3)
sheet_4 = get_crime_set(sheet_4)

In [ ]:
sheet = sheet.rename(columns = {'total': 'total_crime'})
sheet_1 = sheet_1.rename(columns = {'total': 'especialy_serious_crime'})
sheet_2 = sheet_2.rename(columns = {'total': 'serious_crime'})
sheet_3 = sheet_3.rename(columns = {'total': 'crime_of_moderaty_severity'})
sheet_4 = sheet_4.rename(columns = {'total': 'minor_crime'})


In [45]:
df_23 = sheet.merge(sheet_1).merge(sheet_2).merge(sheet_3).merge(sheet_4)
total_df = total_df.merge(df_23, on = 'region', how ='left')
total_df


,region,nominal_income,real_income,avg_salary,real_avg_salary,infant_mortality,infant_mortality_per_1000,disabled,child_pregnancy,congenital_anomalies,...,total_crime_x,especialy_serious_crime_x,serious_crime_x,crime_of_moderaty_severity_x,minor_crime_x,total_crime_y,especialy_serious_crime_y,serious_crime_y,crime_of_moderaty_severity_y,minor_crime_y
0,Алтайский край,31142,97.6,36991.0,100.703512,34,3.6,159038,3.3,200.1,...,34061,1107,4719,8072,20163,34061,1107,4719,8072,20163
1,Амурская область,44891,98.9,65104.0,99.508164,10,2.8,57024,4.4,370.1,...,13261,545,2540,3392,6784,13261,545,2540,3392,6784
2,Архангельская область без Ненецкого АО,45951,95.1,62683.0,96.487654,17,4.1,76702,13.8,458.9,...,15754,773,2734,4295,7952,15754,773,2734,4295,7952
3,Астраханская область,31084,97.7,45833.0,99.697002,27,5.2,40324,0.0,415.1,...,11787,610,1647,2872,6658,11787,610,1647,2872,6658
4,Белгородская область,41009,99.7,46846.0,99.446808,24,4,174508,0.0,136.2,...,10580,400,1490,2531,6159,10580,400,1490,2531,6159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Чеченская Республика,30335,102.3,33843.0,94.821218,103,6.7,184004,0.0,73.6,...,1511,30,303,336,842,1511,30,303,336,842
81,Чувашская Республика,27206,98.4,40222.0,101.180011,11,2,71701,3.3,487.8,...,11763,1877,1633,2143,6110,11763,1877,1633,2143,6110
82,Чукотский автономный округ,113096,103.3,139873.0,102.01494,3,12,1620,111.5,255.2,...,776,35,146,163,432,776,35,146,163,432
83,Ямало-Ненецкий автономный округ,116257,101.4,130013.0,102.502483,15,4.4,13811,11.3,258.9,...,5506,323,791,1170,3222,5506,323,791,1170,3222


In [ ]:
total_df.to_csv('/content/data/total_df.csv', index = False) # Запишем наш датасет в файл csv

#Вывод:
Итак, мы получили таблицу даннх записанную в виде файла csv. Данные представляют из себя информацию по экономическим и демографическим показателям статистики преимуществено за 2022 год. В основном информация была взята на государственном ресурсе "РОССТАТ", также был использован источник "Если быть точнм".